In [208]:
import urllib.request
from IPython import get_ipython

 
# multithreading av v75_scraping
- Körs med v75_scraping(resultat=False,history=True)<br>
- Parametrar: resultat=True/False, history=True/False<br>
- Input: omg_att_spela_link.csv() med en eler flera omgångar/veckor

Första försöket gick inte särskilt bra.
Gör i stället en separat för en avdelning med en egen url

In [209]:
import pandas as pd
import numpy as np
import os

In [210]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

import concurrent.futures

In [211]:
import fixa_mer_features as ff2
import time

In [212]:
def get_webdriver(headless=True):
    # instance of Options class allows
    # us to configure Headless Chrome
    options = Options()
    if headless:
        # this parameter tells Chrome that
        # it should be run without UI (Headless)
        options.headless = True
        
        # the following seems to be necessary for headless to work
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
        options.add_argument('user-agent={0}'.format(user_agent))
        options.add_argument("--window-size=1920x1080")
        
    # initializing webdriver for Chrome with our options

    driver = webdriver.Chrome(
        executable_path='C:\\Users\peter\\Documents\\MyProjects\\gecko\\chromedriver.exe', options=options)
    return driver


In [213]:
def städa_och_rensa(df, history):

    if history:
        for col in ['h1_dat','h2_dat','h3_dat','h4_dat','h5_dat']:
            df[col] = df[col].astype(str)
    
    före = len(df)
    df = ff2.fixa_mer_features(df, history)
    efter = len(df)
    if före-efter != 0:
        print('rensade totalt bort', före-efter,
              'hästar i städa_och_rensa. Från', före, 'till', efter)
    return df

In [214]:
def scrape_utdelning(the_driver):
    # gammal lösn: utdelningar = the_driver.find_elements_by_class_name("css-mxas0-Payouts-styles--amount")
    utdelningar = the_driver.find_elements(By.CLASS_NAME, "css-fu45i8-Payouts-styles--amount")
    utd7 = utdelningar[0].text.replace(' ', '')
    utd7 = utd7.replace('kr', '')
    utd7 = utd7.replace('Jackpot', '0')
    utd6 = utdelningar[1].text.replace(' ', '')
    utd6 = utd6.replace('kr', '')
    utd6 = utd6.replace('Jackpot', '0')
    utd5 = utdelningar[2].text.replace(' ', '')
    utd5 = utd5.replace('kr', '')
    utd5 = utd5.replace('Jackpot', '0')
    return int(utd7), int(utd6), int(utd5)

anpassa kolumnerna som skall visas

In [215]:
def anpassa_kolumner(driver_s):
    sl = driver_s.find_elements(By.CLASS_NAME, "startlist")
    # print('sl all text\n', sl[0].text)
    # print('sl end')
    buts = sl[0].find_elements(
        By.CLASS_NAME, "css-eugx3a-startlistoptionsview-styles--configButton-Button--buttonComponent")
    buts[0].click()
    time.sleep(1)
    print('klickade på', buts[0].text)

    # tics = driver_s.find_elements_by_class_name("css-1hngy38-Checkbox-styles--label")
    WebDriverWait(driver_s, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'css-1hngy38-Checkbox-styles--label')))
    tics = driver_s.find_elements(By.CLASS_NAME,"css-1hngy38-Checkbox-styles--label")
    driver_s.implicitly_wait(5)     # seconds
    print('len tics',len(tics))
    flag1 = flag2 = flag3 = flag4 = flag5 = flag6 = flag7 = flag8 = flag9 = True
    for t in tics:
        if not (flag1 or flag2 or flag3 or flag4 or flag5 or flag6 or flag7 or flag8 or flag9):
            print('anpassa klar - break')
            break
        if t.text == '':
            continue
        if flag1 and t.text == 'VÄRMNINGS­VIDEO':
            # WebDriverWait(driver_s, 10).until(EC.presence_of_element_located((By.NAME, t.NAME))).click()
            t.click()
            flag1 = False
            print('värmn')
        elif flag2 and t.text == 'UTÖKA ALLA STARTLISTOR SAMTIDIGT':
            t.click()
            print('utöka')
            flag2 = False
        elif flag3 and t.text == 'TIPSKOMMENTARER':
            print('tips')
            t.click()
            flag3 = False
        elif flag4 and ('LOPPKOMMENTARER' in t.text):
            print('komment')
            t.click()
            flag4 = False
        elif flag5 and t.text == 'KR/START':
            print('kr')
            t.click()
            flag5 = False
        elif flag6 and t.text == 'DISTANS OCH SPÅR':
            # t.click()
            if t.is_selected():
                print('distans och spår är redan valt')
                flag6 = False
            else:
                print(t.text, 'ej selected ännu')
            WebDriverWait(t, 10).until(EC.element_to_be_clickable(t))
            t.click()
            if t.is_selected():
                print(t.text+' är korrekt')
                flag6 = False
            else:
                print(t.text+' är fortfarande fel')
                flag6 = True
        
        elif flag7 and t.text == 'V-ODDS':
            # t.click()
            print('hoppar över voods click (verkar vara förifyllt')
            flag7 = False
        elif flag8 and t.text == 'P-ODDS':
            # print('podds')
            t.click()
            flag8 = False
        elif flag9 and t.text == 'HÄSTENS KÖN & ÅLDER':
            print('kön')
            t.click()
            flag9 = False
    if flag6:        
        print('Prova name '+'checkbox-ageAndSex')
        chkbx = driver_s.find_elements(By.NAME, 'checkbox-ageAndSex')[0]
    
        if chkbx.is_selected():
            print(chkbx.text+' är korrekt valt')
        else:
            print(chkbx.text, ' ej selected ännu')
            chkbx.click()
        if chkbx.is_selected():
            print(chkbx.text+' är korrekt')
    
    ## Tryck på Spara-knappen ##
    driver_s.implicitly_wait(5)     # seconds
    save_button = driver_s.find_elements(By.CLASS_NAME,
        "css-1fh4n7y-Button-styles--root-PrimaryButton-styles--root-StartlistDisplayOptionsDialog-styles--saveButton-PrimaryButton--PrimaryButton-StartlistDisplayOptionsDialog-styles--saveButton")
    save_button[0].click()
    print('efter click Spara')


hämta utdelnings-info och spara på fil

In [216]:
def utdelning(driver_r, dat, bana):
    utd7, utd6, utd5 = scrape_utdelning(driver_r)
    assert utd7 > utd6 or utd7==0, '7 rätt skall ge mer pengar än 6 rätt'
    assert utd6 > utd5 or utd6 == 0, '6 rätt skall ge mer pengar än 5 rätt'
    print(f'utdelning: {utd7}, {utd6}, {utd5}')
    utd_file = 'C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\utdelning.csv'
    utdelning = pd.read_csv(utd_file)
    utd = pd.DataFrame([[dat, bana, utd7, utd6, utd5]])
    utd.columns = utdelning.columns
    utdelning = pd.concat([utdelning, utd], ignore_index=True)
    utdelning.drop_duplicates(['datum'], inplace=True)
    utdelning.sort_values(by=['datum'], inplace=True)
    utdelning.reset_index(drop=True, inplace=True)
    utdelning.to_csv(utd_file, index=False)

returnerar en lista av placeringar i startnr-ordning

In [217]:
def inkludera_resultat(res_avd, anr):
    res_rader = res_avd.find_elements(By.CLASS_NAME,
        "startlist__row")  # alla rader i loppet
    res_startnr = res_avd.find_elements(By.CLASS_NAME,
        "css-1jc4209-horseview-styles--startNumber")  # alla startnr
    d = {'plac': [], 'startnr': []}
    for nr, rad in enumerate(res_rader):
        plac = rad.text.split(' ')[0]
        d['plac'].append(plac)
        d['startnr'].append(res_startnr[nr].text)
    temp_df = pd.DataFrame(d)
    temp_df['startnr'] = temp_df.startnr.astype(int)
    temp_df.sort_values(by='startnr', ascending=True, inplace=True)
    return temp_df.plac.to_list()

In [218]:
### Priser ###

def get_priser(driver):
    priselement = driver.find_elements(
        By.CLASS_NAME, 'css-1lnkuf6-startlistraceinfodetails-styles--infoContainer')
    NOK, EUR = False, False
    for p in priselement:
        if 'EUR' in p.text:
            EUR = True
            break
        elif 'NOK' in p.text:
            NOK = True
            break
    print('EUR:', EUR, 'NOK:', NOK)
    # alla lopps priser
    priser = [p.text for p in priselement if 'Pris:' in p.text]
    if len(priser) == 0:
        priser = [p.text for p in priselement if 'åriga' not in p.text and (
            'EUR' in p.text or 'NOK' in p.text)]  # alla lopps priser
    print('priser', len(priser))
    uppdaterade_priser=[]
    for anr in range(len(priser)):
        if EUR:
            pris = priser[anr].split('-')[0] + '0'    # Euro -> SEK
        elif NOK:
            pris = priser[anr].split('-')[0].replace('.', '')  # NOK -> SEK
        else:
            pris = priser[anr].split('-')[0].split(' ')[1]   
        uppdaterade_priser.append(pris)
   
    return uppdaterade_priser

### url list ###
def get_url_list(driver):
    # alla lopp
    get_url = driver.current_url
    ## cut out the last part of the url
    part1_url = get_url[:get_url.rfind('/')]
    url_list = [part1_url+'/avd'+str(i) for i in range(1, 8)]
    return url_list


scraping av en häst

In [219]:
def en_häst(vd, r, rad, history, driver_avd):
    # en rad är en häst     
        # kusk måste tas på hästnivå (pga att driver-col finns också i hist)
    vd['kusk'].append(rad.find_elements(By.CLASS_NAME, "driver-col")[0].text)
    vd['ålder'].append(rad.find_elements(By.CLASS_NAME, "horse-age")[0].text)
    vd['kön'].append(rad.find_elements(By.CLASS_NAME, "horse-sex")[0].text)
    vd['kr'].append(rad.find_elements(By.CLASS_NAME, "earningsPerStart-col")[0].text)  # kr/start i lopp 1 utan vd
    # dist och spår i lopp 1 utan rubrik
    dist_sp = rad.find_elements(By.CLASS_NAME, "postPositionAndDistance-col")
    assert len(dist_sp) > 0, f'dist_sp ej len>0 len={len(dist_sp)} {dist_sp}'
    vd['dist'].append(dist_sp[0].text.split(':')[0])
    vd['spår'].append(dist_sp[0].text.split(':')[1])

    if history:
        ## history från startlistan ##
        hist = driver_avd.find_elements(By.CLASS_NAME, "start-info-panel")  # all history för loppet
        h_dates = hist[r].find_elements(By.CLASS_NAME, 'date-col')[1:]
        h_kuskar = hist[r].find_elements(By.CLASS_NAME,'driver-col')[1:]
        h_banor = hist[r].find_elements(By.CLASS_NAME, 'track-col')[1:]
        h_plac = hist[r].find_elements(By.CLASS_NAME, 'place-col')[1:]  # obs varannan rad (10 rader)
        h_plac = h_plac[::2]   # ta ut varannat värde
        h_dist = hist[r].find_elements(By.CLASS_NAME,'distance-col')[1:]
        h_spår = hist[r].find_elements(By.CLASS_NAME,'position-col')[1:]
        h_kmtid = hist[r].find_elements(By.CLASS_NAME, 'kmTime-col')[1:]
        h_odds = hist[r].find_elements(By.CLASS_NAME, 'odds-col')[1:]
        h_pris = hist[r].find_elements(By.CLASS_NAME,'firstPrize-col')[1:]
        for h, d in enumerate(h_dates):
            fld = 'h'+str(h+1)+'_'
            dtext = d.text
            vd[fld+'dat'].append(dtext)
            vd[fld+'bana'].append(h_banor[h].text)
            vd[fld+'kusk'].append(h_kuskar[h].text)
            vd[fld+'plac'].append(h_plac[h].text)
            vd[fld+'dist'].append(h_dist[h].text)
            vd[fld+'spår'].append(h_spår[h].text)
            vd[fld+'kmtid'].append(h_kmtid[h].text)
            vd[fld+'odds'].append(h_odds[h].text)
            vd[fld+'pris'].append(h_pris[h].text)
        if len(h_dates) < 5:
            # Duplicera hist_4
            fld = 'h'+str(5)+'_'
            h = 3
            vd[fld+'dat'].append(dtext)
            vd[fld+'bana'].append(h_banor[h].text)
            vd[fld+'kusk'].append(h_kuskar[h].text)
            vd[fld+'plac'].append(h_plac[h].text)
            vd[fld+'dist'].append(h_dist[h].text)
            vd[fld+'spår'].append(h_spår[h].text)
            vd[fld+'kmtid'].append(h_kmtid[h].text)
            vd[fld+'odds'].append(h_odds[h].text)
            vd[fld+'pris'].append(h_pris[h].text)
    return vd

scraping av en avd = ett lopp

In [220]:
# scrape one avd (=lopp)
def ett_lopp(anr,driver_avd, comb, priser, result_tab ,driver_r,history, datum):
    
    print(f'Scraping avd {anr+1}')
   
    vd = {'datum': [], 'bana': [], 'avd': [], 'startnr': [], 'häst': [], 'ålder': [], 'kön': [], 'kusk': [], 'lopp_dist': [],
             'start': [], 'dist': [], 'pris': [], 'spår': [], 'streck': [], 'vodds': [], 'podds': [], 'kr': [], }
    if history:
        vd.update(
            {'h1_dat': [], 'h1_bana': [], 'h1_kusk': [], 'h1_plac': [], 'h1_dist': [], 'h1_spår': [], 'h1_odds': [], 'h1_pris': [], 'h1_kmtid': [],
             'h2_dat': [], 'h2_bana': [], 'h2_kusk': [], 'h2_plac': [], 'h2_dist': [], 'h2_spår': [], 'h2_odds': [], 'h2_pris': [], 'h2_kmtid': [],
             'h3_dat': [], 'h3_bana': [], 'h3_kusk': [], 'h3_plac': [], 'h3_dist': [], 'h3_spår': [], 'h3_odds': [], 'h3_pris': [], 'h3_kmtid': [],
             'h4_dat': [], 'h4_bana': [], 'h4_kusk': [], 'h4_plac': [], 'h4_dist': [], 'h4_spår': [], 'h4_odds': [], 'h4_pris': [], 'h4_kmtid': [],
             'h5_dat': [], 'h5_bana': [], 'h5_kusk': [], 'h5_plac': [], 'h5_dist': [], 'h5_spår': [], 'h5_odds': [], 'h5_pris': [], 'h5_kmtid': [],
             })
    my_vdict = vd.copy()
    
    bana = comb[anr].text.split('\n')[0]
    lopp_dist = comb[anr].text.split('\n')[1].split(' ')[0][:-1]
    start = comb[anr].text.split('\n')[1].split(' ')[1]
    pris = priser[anr]
    
    # alla hästar/kön/åldet i loppet
    names = driver_avd.find_elements(By.CLASS_NAME, "horse-name")
    voddss = driver_avd.find_elements(By.CLASS_NAME, "vOdds-col")[1:]  # vodds i loppet utan rubrik
    poddss = driver_avd.find_elements(By.CLASS_NAME, "pOdds-col")[1:]  # podds i loppet utan rubrik
    rader = driver_avd.find_elements(By.CLASS_NAME, "startlist__row")  # alla rader i loppet
    strecks = driver_avd.find_elements(By.CLASS_NAME, "betDistribution-col")[1:]  # streck i loppet  utan rubrik
    print(f'avd {anr+1} ant (names,vodds,podds,rader,streck (vd len(häst)))', len(names), len(voddss), len(poddss), len(strecks), len(vd["häst"]))
    if driver_r:
        # resultat från resultatsidan
        res_avd = result_tab[anr]
        # placeringar sorterade efter startnr för en avd
        places = inkludera_resultat(res_avd, anr+1)
        vd['plac'] += places    # konkatenera listorna
    
    # print('AVD', anr+1, bana, lopp_dist, start)    #, end=' ')
    
    for r, rad in enumerate(rader):
        vd['datum'].append(datum)
        vd['bana'].append(bana)
        vd['avd'].append(anr+1)
        vd['startnr'].append(r+1)
        vd['häst'].append(names[r].text)
        vd['start'].append(start)
        vd['lopp_dist'].append(lopp_dist)
        vd['pris'].append(pris)
        vd['vodds'].append(voddss[r].text)  # vodds i lopp 1 utan rubrik
        vd['podds'].append(poddss[r].text)  # podds i lopp 1 utan rubrik
        # streck i lopp 1  utan rubrik
        vd['streck'].append(strecks[r].text)
    
    # vd = en_häst(vd, r, rad, history, driver_avd)
        
    print(f'start häst')
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        # map excutor driver_avd_tab with ett_lopp(i,driver_avd_tab[i], comb, priser, result_tab, vdict, datum)
        future_list = [executor.submit(en_häst, vd, r, rad, history, driver_avd) for r, rad in enumerate(rader)]
    print(f'end häst')
    
    for future in concurrent.futures.as_completed(future_list):
        try:
            data = future.result()
            # append data to my_vdict
            my_vdict= {k: my_vdict[k] + data[k] for k in data.keys()}
            # print(my_vdict['avd'])
        except Exception as exc:
            print('%r in "en_häst" generated an exception: %s' % (future, exc))
            
    # assert False, "take a break"
    
    # temp=pd.DataFrame(vd)
    # temp.to_csv('vdict'+str(anr)+'.csv')
    return vd
    
#


In [221]:
if False:
    bana = com[an].text.split('\n')[0]
    lopp_dist = com[an].text.split('\n')[1].split(' ')[0][:-1]
    start = com[an].text.split('\n')[1].split(' ')[1]
    pris = prise[an]
    vdict['datum'] = []
    # alla hästar/kön/åldet i loppet
    names = driver_av.find_elements(By.CLASS_NAME, "horse-name")
    voddss = driver_av.find_elements(
        By.CLASS_NAME, "vOdds-col")[1:]  # vodds i loppet utan rubrik
    poddss = driver_av.find_elements(
        By.CLASS_NAME, "pOdds-col")[1:]  # podds i loppet utan rubrik
    rader = driver_av.find_elements(
        By.CLASS_NAME, "startlist__row")  # alla rader i loppet
    strecks = driver_av.find_elements(
        By.CLASS_NAME, "betDistribution-col")[1:]  # streck i loppet  utan rubrik
    print(f'avd {an+1} ant (names,vodds,podds,rader,streck (vdict len(häst)))',
            len(names), len(voddss), len(poddss), len(strecks), len(vdict["häst"]))

        # print('AVD', anr+1, bana, lopp_dist, start)    #, end=' ')
    for r, rad in enumerate(rader):
        # en rad är en häst
        vdict['datum'].append(datu)
        vdict['bana'].append(bana)
        vdict['avd'].append(an+1)
        vdict['startnr'].append(r+1)
        vdict['häst'].append(names[r].text)
        vdict['start'].append(start)
        vdict['lopp_dist'].append(lopp_dist)
        vdict['pris'].append(pris)
        vdict['vodds'].append(voddss[r].text)  # vodds i lopp 1 utan rubrik
        vdict['podds'].append(poddss[r].text)  # podds i lopp 1 utan rubrik
        # streck i lopp 1  utan rubrik
        vdict['streck'].append(strecks[r].text)
        # kusk måste tas på hästnivå (pga att driver-col finns också i hist)
        vdict['kusk'].append(rad.find_elements(
            By.CLASS_NAME, "driver-col")[0].text)
        vdict['ålder'].append(rad.find_elements(
            By.CLASS_NAME, "horse-age")[0].text)
        vdict['kön'].append(rad.find_elements(
            By.CLASS_NAME, "horse-sex")[0].text)
        vdict['kr'].append(rad.find_elements(
            By.CLASS_NAME, "earningsPerStart-col")[0].text)  # kr/start i lopp 1 utan rubrik
        # dist och spår i lopp 1 utan rubrik
        dist_sp = rad.find_elements(By.CLASS_NAME, "postPositionAndDistance-col")
        assert len(dist_sp) > 0, f'dist_sp ej len>0 len={len(dist_sp)} {dist_sp}'
        
        vdict['dist'].append(dist_sp[0].text.split(':')[0])
        vdict['spår'].append(dist_sp[0].text.split(':')[1])
        
        print(an+1, 'len(vdict["häst"])', len(vdict['häst']))

    len(vdict['bana'])


In [222]:
if False:
    # print length of all items in vdict
    for k in vdict:
        print(k, len(vdict[k]))
        vdict[k] = []

scraping för en omgång

In [223]:
def en_omg(omg, driver_list, driver_r, history, first_time):  # get data from web site for one V75 omgång
    vdict = {'datum': [], 'bana': [], 'avd': [], 'startnr': [], 'häst': [], 'ålder': [], 'kön': [], 'kusk': [], 'lopp_dist': [],
             'start': [], 'dist': [], 'pris': [], 'spår': [], 'streck': [], 'vodds': [], 'podds': [], 'kr': [], }
    if history:
        vdict.update(
            {'h1_dat': [], 'h1_bana': [], 'h1_kusk': [], 'h1_plac': [], 'h1_dist': [], 'h1_spår': [], 'h1_odds': [], 'h1_pris': [], 'h1_kmtid': [],
             'h2_dat': [], 'h2_bana': [], 'h2_kusk': [], 'h2_plac': [], 'h2_dist': [], 'h2_spår': [], 'h2_odds': [], 'h2_pris': [], 'h2_kmtid': [],
             'h3_dat': [], 'h3_bana': [], 'h3_kusk': [], 'h3_plac': [], 'h3_dist': [], 'h3_spår': [], 'h3_odds': [], 'h3_pris': [], 'h3_kmtid': [],
             'h4_dat': [], 'h4_bana': [], 'h4_kusk': [], 'h4_plac': [], 'h4_dist': [], 'h4_spår': [], 'h4_odds': [], 'h4_pris': [], 'h4_kmtid': [],
             'h5_dat': [], 'h5_bana': [], 'h5_kusk': [], 'h5_plac': [], 'h5_dist': [], 'h5_spår': [], 'h5_odds': [], 'h5_pris': [], 'h5_kmtid': [],
             })
        
    my_vdict = vdict.copy()
    datum = omg.split('spel/')[1][0:10]
    print(f'omgång:', datum)
    
    driver_s = driver_list[0]      
    driver_s.get(omg)              # öppna startlista för omgången
    driver_s.implicitly_wait(5)    # seconds
    
    if first_time:  # we only need to run anpassa_kolumner once in this session
        # första gången startlista
        # ok till första popup godkänn kakor
        WebDriverWait(driver_s, 10).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler")))
        driver_s.find_element(By.ID, "onetrust-accept-btn-handler").click()
        driver_s.fullscreen_window()
        driver_s.find_elements(By.CLASS_NAME,"race-info-toggle")[1].click()  # prissummor mm
        anpassa_kolumner(driver_s)
        
    result_tab=None
    if driver_r:
        vdict.update({'plac': []})
        result_tab = driver_r.find_elements(By.CLASS_NAME,
                                            'game-table')[:]  # alla lopp med resultatordning
        if len(result_tab) == 8:
            result_tab = result_tab[1:]
        print('ant resultat', len(result_tab))
    
    
    priser = get_priser(driver_s)
        
    print('priser', priser)
    print('Ant priser', len(priser))
        
    # avd_url_list=get_url_list(driver_s)

    driver_avd_tab = driver_s.find_elements(By.CLASS_NAME, 'game-table')[:]  # drivers för alla lopp
    if len(driver_avd_tab) == 8:
        driver_avd_tab = driver_avd_tab[1:]
    print('ant lopp', len(driver_avd_tab))

    comb = driver_s.find_elements(By.CLASS_NAME, 'race-combined-info')  # alla bana,dist,start

    # de häst-relaterade värdena som inte kan bli 'missing' tar jag direkt på loppnivå)
    print(f'start executor')
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        # map excutor driver_avd_tab with ett_lopp(i,driver_avd_tab[i], comb, priser, result_tab, vdict, datum)
        future_list = [executor.submit(ett_lopp, i, driver_avd_tab[i], comb, priser, result_tab, driver_r, history, datum) for i in range(len(driver_avd_tab))]
    print(f'end executor')
    
    for future in concurrent.futures.as_completed(future_list):
        try:
            data = future.result()
            # append data to my_vdict
            my_vdict= {k: my_vdict[k] + data[k] for k in data.keys()}
            # print(my_vdict['avd'])
        except Exception as exc:
            print('%r in "en_omg" generated an exception: %s' % (future, exc))
    
    bana = comb[0].text.split('\n')[0]
    return my_vdict, bana


In [224]:
# vdct={}
# vdct1 = {'datum': [1], 'bana': [1], 'avd': [1], 'startnr': [1], 'häst': [1]}
# vdct2 = {'datum': [2], 'bana': [2], 'avd': [2], 'startnr': [2], 'häst': [2]}
# # for k in vdct1.keys():
# #     vdct[k] = vdct1[k] + vdct2[k]

# vdct.update(vdct1)

# # dict comprahension
# vdct = {k: vdct[k] + vdct2[k] for k in vdct2.keys()}
# vdct


## Starta Här!
Öppnar v75 sidorna (startlist plus ev resultat) och samlar all info per avd/häst

In [225]:
def v75_scraping(history=False, headless=True, driver_r=False):
    ### Hela loopen med alla lopp i alla veckor i omg_df ###
    start_time = time.perf_counter()
    omglinks_df = pd.read_csv('C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\omg_att_spela_link.csv')
    df_alla_omgångar = pd.DataFrame()
    if driver_r:
        driver_r = get_webdriver(headless)  # get web driver for results
        
    driver_list=[]
    print('start get webdrivers')
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        # map excutor  driver_list and get_webdriver
        driver_list = list(executor.map(get_webdriver, [headless]*7))
    print('end get webdrivers')
        
    for enum, omg in enumerate(omglinks_df.Link.values):
        # om resultat
        if driver_r:
            driver_r.get(omg+'/resultat')   # öppna resultat
            driver_r.implicitly_wait(5)     # seconds
            if enum == 0:
                # ok till första popup om kakor
                driver_r.find_element(By.ID,"onetrust-accept-btn-handler").click()
                driver_r.fullscreen_window()

        # scraping av en omgång
        komplett, bana = en_omg(omg, driver_list, driver_r, history, enum==0)
        temp_df = pd.DataFrame(komplett)
        df_alla_omgångar = pd.concat([df_alla_omgångar, temp_df])

        # temp_df.to_csv('df_temp.csv', index=False)
        # df_alla_omgångar.to_csv('df_alla_omgångar.csv', index=False)
        # utdelning
        if driver_r:
            datum = omg.split('spel/')[1][0:10]
            utdelning(driver_r, datum, bana)  # utdelning för denna omgång


    print('start quit drivers')   
    with concurrent.futures.ThreadPoolExecutor(max_workers=7) as executor:
        # map excutor  driver_list and quit
        executor.map(lambda driver: driver.quit(), driver_list)
    print('end quit drivers')
        
    if driver_r:
        driver_r.quit()
        
    strukna = df_alla_omgångar.loc[df_alla_omgångar.vodds == 'EJ', :].copy()  # Ta bort all strukna
    
    df_alla_omgångar = städa_och_rensa(df_alla_omgångar, history)

    print('\ndet tog', round(time.perf_counter() - start_time, 3), 'sekunder')
    
    return df_alla_omgångar, strukna


In [226]:
# driver_s,driver_r=None,None
# headless=True
# history=False
# resultat=False
# omg_df = pd.read_csv(
#     'C:\\Users\\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel\\omg_att_spela_link.csv')
# alla_omg_df = pd.DataFrame()  # alla omgångar data
# if not driver_r:
#     if resultat:
#         driver_r = get_webdriver(headless)  # get web driver for results
#     else:
#         driver_r = None
# if not driver_s:
#     driver_s = get_webdriver(headless)  # web driver för startlista
# for enum, omg in enumerate(omg_df.Link.values):
#     print(f'omgång {enum+1}:', omg)
#     datum = omg.split('spel/')[1][0:10]
#     driver_s.get(omg)              # öppna startlista
#     driver_s.implicitly_wait(5)     # seconds
#     if enum == 0:  # första gången startlista
#         # ok till första popup om kakor
#         driver_s.find_element(By.ID,"onetrust-accept-btn-handler").click()
#         driver_s.fullscreen_window()
#         driver_s.implicitly_wait(5)     # seconds
#         driver_s.find_elements(By.CLASS_NAME,
#             "race-info-toggle")[1].click()  # prissummor mm
#         driver_s.implicitly_wait(5)     # seconds
#         anpassa(driver_s)

#     # resultat
#     if resultat:
#         driver_r.get(omg+'/resultat')   # öppna resultat
#         driver_r.implicitly_wait(5)     # seconds
#         if enum == 0:
#             # ok till första popup om kakor
#             driver_r.find_element(By.ID,"onetrust-accept-btn-handler").click()
#             driver_r.fullscreen_window()

#     # scraping
#     komplett, bana = do_scraping(driver_s, driver_r, history, datum)
#     temp_df = pd.DataFrame(komplett)
#     alla_omg_df = pd.concat([alla_omg_df, temp_df])

#     # utdelning
#     if resultat:
#         utdelning(driver_r, datum, bana)  # utdelning för denna omgång

# # för att göra tester och debugging efteråt
# # global df_spara
# # df_spara = df.copy()
# ###
# driver_s.quit()
# if resultat:
#     driver_r.quit()
# strukna = alla_omg_df.loc[alla_omg_df.vodds == 'EJ', :].copy()  # Ta bort all strukna
# temp_df = städa_och_rensa(temp_df, history)

kör igång

In [229]:
if __name__ == '__main__':
    print('START GOING')
    df,_=v75_scraping(history=True, headless=True, driver_r=False)
print('END')

START GOING
start get webdrivers


C:\Users\peter\AppData\Local\Temp/ipykernel_1080/1955018772.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


end get webdrivers
omgång: 2022-04-30
klickade på Anpassa
len tics 27
kr
hoppar över voods click (verkar vara förifyllt
DISTANS OCH SPÅR ej selected ännu
DISTANS OCH SPÅR är fortfarande fel
värmn
kön
utöka
komment
Prova name checkbox-ageAndSex
 är korrekt valt
 är korrekt
efter click Spara
EUR: False NOK: False
priser 7
priser ['110.000', '110.000', '110.000', '110.000', '110.000', '150.000', '400.000']
Ant priser 7
ant lopp 7
start executor
Scraping avd 1
Scraping avd 2
Scraping avd 3
Scraping avd 4
Scraping avd 5
Scraping avd 6
Scraping avd 7
avd 1 ant (names,vodds,podds,rader,streck (vd len(häst))) 12 12 12 12 0
avd 2 ant (names,vodds,podds,rader,streck (vd len(häst))) 12 12 12 12 0
avd 3 ant (names,vodds,podds,rader,streck (vd len(häst))) 11 11 11 11 0
avd 4 ant (names,vodds,podds,rader,streck (vd len(häst))) 15 15 15 15 0
avd 5 ant (names,vodds,podds,rader,streck (vd len(häst))) 12 12 12 12 0
avd 6 ant (names,vodds,podds,rader,streck (vd len(häst))) 10 10 10 10 0
avd 7 ant (names,

In [228]:
df
# dfAll = pd.read_csv("../all_data.csv")[['datum','avd','häst','pris','streck']]
# len(dfAll[dfAll.pris < 1000].datum.unique())

,datum,bana,avd,startnr,häst,ålder,kön,kusk,lopp_dist,start,dist,pris,spår,streck,vodds,podds,kr,bins
0,2022-04-30,ÖREBRO,1,1,CRANK,6,v,Kenneth Haugstad,2140.0,1,2140.0,110.0,1.0,1.0,75.88,9.63,13782.0,29
1,2022-04-30,ÖREBRO,1,2,HUMBLE MAN,4,v,Robert Bergh,2140.0,1,2140.0,110.0,2.0,3.0,23.14,6.46,33541.0,19
2,2022-04-30,ÖREBRO,1,3,BROADWAY SUN,5,h,Rikard N Skoglund,2140.0,1,2140.0,110.0,3.0,11.0,14.48,3.44,24215.0,14
3,2022-04-30,ÖREBRO,1,4,HEFNER AM,5,v,Ulf Ohlsson,2140.0,1,2140.0,110.0,4.0,9.0,8.38,2.88,21372.0,9
4,2022-04-30,ÖREBRO,1,5,BRUNELLO HALL,6,h,Emilia Leo,2140.0,1,2140.0,110.0,5.0,2.0,26.06,5.22,14117.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2022-04-30,ÖREBRO,7,11,ORACLE TILE,6,h,Jorma Kontio,3140.0,1,3160.0,400.0,6.0,2.0,28.40,6.56,36742.0,21
81,2022-04-30,ÖREBRO,7,12,SELECTED NEWS,5,h,Magnus A Djuse,3140.0,1,3180.0,400.0,1.0,3.0,14.71,4.05,73653.0,14
82,2022-04-30,ÖREBRO,7,13,MONI VIKING,9,h,Björn Goop,3140.0,1,3180.0,400.0,2.0,40.0,2.60,1.53,142341.0,1
83,2022-04-30,ÖREBRO,7,14,ELECTRICAL STORM,8,v,Mattias Djuse,3140.0,1,3180.0,400.0,3.0,1.0,30.55,4.86,24331.0,21
